In [9]:
import tkinter as tk
from tkinter import messagebox
import random
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import re
import time  # Import the time module

# Global Variables
sender_attempts = 0
email_attempts = 0
otp_attempts = 0
recipient_email_attempts = 0
max_attempts = 3
otp = None
otp_generated_time = None  # Timestamp when OTP was generated
otp_expiration_time = 180  # OTP expiration time in seconds (3 minutes)
smtp_server = 'smtp.gmail.com'
smtp_port = 465
sender_verified = False

def generate_otp():
    """
    Generates a random 6-digit One-Time Password (OTP).

    Returns:
        int: A 6-digit integer representing the generated OTP, or None if an error occurs.
    """
    try:
        # Generate a random integer between 100000 and 999999 inclusive.
        return random.randint(100000, 999999)
    except Exception as e:
        # Print an error message if an exception occurs during OTP generation.
        print(f"Error generating OTP: {e}")
        # Return None to indicate that OTP generation failed.
        return None

def send_otp_via_email(otp, sender_email_address, sender_password, recipient_email_address):
    """
    Sends an OTP code via email to the specified recipient.

    Args:
        otp (int): The OTP code to be sent.
        sender_email_address (str): The email address of the sender.
        sender_password (str): The password for the sender's email account.
        recipient_email_address (str): The email address of the recipient.

    Returns:
        tuple: A tuple where the first element is a boolean indicating success (True) or failure (False),
               and the second element is an error object if there was a failure, otherwise None.
    """
    try:
        # Create a MIME multipart message object to handle email components.
        msg = MIMEMultipart()
        
        # Set the email subject line.
        msg['Subject'] = 'Your OTP Code'
        
        # Set the sender's email address.
        msg['From'] = sender_email_address
        
        # Set the recipient's email address.
        msg['To'] = recipient_email_address
        
        # Attach the OTP message body to the email. Include a note that the OTP is valid for 3 minutes.
        msg.attach(MIMEText(f"Your OTP is {otp}. This code is only valid for 3 minutes.", 'plain'))

        # Connect to the SMTP server using SSL for secure communication.
        with smtplib.SMTP_SSL(smtp_server, smtp_port) as server:
            # Login to the SMTP server using the sender's email address and password.
            server.login(sender_email_address, sender_password)
            
            # Send the email with the OTP to the recipient.
            server.sendmail(msg['From'], [msg['To']], msg.as_string())
        
        # Print a success message if the OTP is sent without errors.
        print("OTP sent successfully.")
        return True, None  # Indicate success and no error.

    except smtplib.SMTPException as smtp_error:
        # Catch and handle SMTP-specific errors.
        print(f"SMTP error occurred: {smtp_error}")
        return False, smtp_error  # Indicate failure and return the SMTP error.

    except Exception as e:
        # Catch and handle any other exceptions that occur during the process.
        print(f"Failed to send OTP: {e}")
        return False, e  # Indicate failure and return the general error.

def validate_email(email):
    """
    Validates the given email address to ensure it conforms to standard email format rules.

    Args:
        email (str): The email address to be validated.

    Returns:
        bool: True if the email address is valid, False otherwise.
    """
    try:
        # Check if the email contains exactly one '@' symbol.
        if email.count('@') != 1:
            raise ValueError("Email must contain exactly one '@' symbol.")
        
        # Split the email into local part and domain part.
        local_part, domain_part = email.split('@')
        
        # Validate the local part of the email.
        if not local_part or local_part.startswith('.') or local_part.endswith('.') or '..' in local_part:
            raise ValueError("Invalid local part of the email.")
        
        # Check if the local part contains only valid characters.
        if not re.match(r'^[a-zA-Z0-9_.+-]+$', local_part):
            raise ValueError("Local part contains invalid characters.")
        
        # Validate the domain part of the email.
        if not domain_part or domain_part.startswith('-') or domain_part.endswith('-') or '..' in domain_part:
            raise ValueError("Invalid domain part of the email.")
        
        # Check if the domain part matches the standard domain format.
        if not re.match(r'^[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', domain_part):
            raise ValueError("Invalid domain part of the email.")
        
        # Ensure the total length of the email address does not exceed 254 characters.
        if len(email) > 254:
            raise ValueError("Email address exceeds the maximum length of 254 characters.")
        
        # If all checks pass, the email is valid.
        return True
    
    except ValueError as ve:
        # Catch and print validation-specific errors.
        print(f"Validation Error: {ve}")
        return False
    
    except Exception as e:
        # Catch and print any unexpected errors.
        print(f"Unexpected Error: {e}")
        return False

def verify_otp(generated_otp, entered_otp):
    """
    Verifies if the entered OTP matches the generated OTP.

    Args:
        generated_otp (int): The OTP that was originally generated and sent.
        entered_otp (int): The OTP entered by the user for verification.

    Returns:
        bool: True if the entered OTP matches the generated OTP, False otherwise.
    """
    # Check if the entered OTP is equal to the generated OTP.
    return generated_otp == entered_otp

def is_otp_expired():
    """
    Checks if the OTP has expired based on the time elapsed since it was generated.

    Returns:
        bool: True if the OTP has expired or if no OTP has been generated, False otherwise.
    """
    global otp_generated_time, otp_expiration_time

    # Check if OTP has been generated. If not, it is considered expired.
    if otp_generated_time is None:
        return True  # No OTP generated yet, so it's considered expired.
    
    # Calculate the time elapsed since OTP generation.
    elapsed_time = time.time() - otp_generated_time
    
    # Determine if the elapsed time exceeds the OTP expiration time.
    return elapsed_time > otp_expiration_time

def verify_sender_credentials(sender_email, sender_password):
    """
    Verifies the sender's email credentials by attempting to log in to the SMTP server.

    Args:
        sender_email (str): The email address of the sender.
        sender_password (str): The password for the sender's email account.

    Returns:
        tuple: A tuple where the first element is a boolean indicating success (True) or failure (False),
               and the second element is an error object if there was a failure, otherwise None.
    """
    try:
        # Establish a secure connection to the SMTP server using SSL.
        with smtplib.SMTP_SSL(smtp_server, smtp_port) as server:
            # Attempt to log in to the SMTP server with the provided email address and password.
            server.login(sender_email, sender_password)
        
        # If login is successful, return True and None to indicate success.
        return True, None
    
    except smtplib.SMTPException as smtp_error:
        # Catch and handle SMTP-specific errors (e.g., authentication errors).
        return False, smtp_error
    
    except Exception as e:
        # Catch and handle any other unexpected exceptions that occur during the process.
        return False, e

def handle_verify_credentials(sender_email_entry, sender_password_entry, verify_button, send_otp_button, recipient_email_entry, root):
    """
    Handles the verification of the sender's email credentials.

    Args:
        sender_email_entry (tk.Entry): The entry widget for the sender's email address.
        sender_password_entry (tk.Entry): The entry widget for the sender's email password.
        verify_button (tk.Button): The button widget used to initiate credential verification.
        send_otp_button (tk.Button): The button widget used to send the OTP, enabled only after verification.
        recipient_email_entry (tk.Entry): The entry widget for the recipient's email address.
        root (tk.Tk): The main application window.

    Returns:
        None
    """
    global sender_attempts, sender_verified

    # Check if sender credentials have already been verified.
    if sender_verified:
        return  # Exit the function if credentials are already verified.

    # Increment the number of attempts to verify sender credentials.
    sender_attempts += 1

    # Retrieve the email and password entered by the user.
    sender_email = sender_email_entry.get()
    sender_password = sender_password_entry.get()

    # Validate the format of the sender's email address.
    if validate_email(sender_email):
        # Attempt to verify the sender's credentials with the provided email and password.
        is_verified, error = verify_sender_credentials(sender_email, sender_password)
        
        if is_verified:
            # If verification is successful, update global variables and UI elements accordingly.
            sender_verified = True
            sender_attempts = 0
            messagebox.showinfo("Verified", "Sender credentials verified successfully.")
            
            # Disable the verify button and enable the OTP-related buttons.
            verify_button.config(state="disabled")
            send_otp_button.config(state="normal")
            recipient_email_entry.config(state="normal")
            sender_email_entry.config(state="disabled")
            sender_password_entry.config(state="disabled")
        else:
            # If verification fails, show an error message with the number of remaining attempts.
            messagebox.showerror("SMTP Error", f"Failed to verify credentials: {error}. Attempts remaining: {max_attempts - sender_attempts}")
    else:
        # If the email format is invalid, show an error message with the number of remaining attempts.
        messagebox.showerror("Invalid Email", f"Please enter a valid sender email address. Attempts remaining: {max_attempts - sender_attempts}")

    # Check if the maximum number of attempts has been reached.
    if sender_attempts >= max_attempts:
        # If maximum attempts are reached, show an error message and close the application.
        messagebox.showerror("Error", "Maximum attempts to verify sender credentials reached. Exiting the application.")
        root.destroy()  # Close the main application window.

def handle_send_otp(sender_email, sender_password, recipient_email_entry, otp_entry, send_otp_button, submit_otp_button, root):
    """
    Handles the process of sending an OTP to the recipient's email address and updating the UI accordingly.

    Args:
        sender_email (str): The email address of the sender.
        sender_password (str): The password for the sender's email account.
        recipient_email_entry (tk.Entry): The entry widget for the recipient's email address.
        otp_entry (tk.Entry): The entry widget for entering the OTP.
        send_otp_button (tk.Button): The button widget used to send the OTP.
        submit_otp_button (tk.Button): The button widget used to submit the OTP.
        root (tk.Tk): The main application window.

    Returns:
        None
    """
    global otp, recipient_email_attempts, otp_generated_time

    # Retrieve the recipient's email address from the entry widget.
    recipient_email_address = recipient_email_entry.get()
    
    # Validate the format of the recipient's email address.
    if not validate_email(recipient_email_address):
        # Increment the count of invalid attempts for entering the recipient's email.
        recipient_email_attempts += 1
        
        # Display an error message if the email address is invalid.
        messagebox.showerror("Invalid Email", f"Please enter a valid recipient email address. Attempts remaining: {max_attempts - recipient_email_attempts}")
        
        # If the maximum number of attempts is reached, show an error message and exit the application.
        if recipient_email_attempts >= max_attempts:
            messagebox.showerror("Error", "Maximum attempts to enter recipient email reached. Exiting the application.")
            root.destroy()  # Close the main application window.
        return  # Exit the function if the recipient email is invalid.

    # Generate a new OTP.
    otp = generate_otp()
    if otp is None:
        # Display an error message if OTP generation fails.
        messagebox.showerror("Error", "Failed to generate OTP. Exiting.")
        root.destroy()  # Close the main application window.
        return  # Exit the function if OTP generation fails.

    # Record the current time to track when the OTP was generated.
    otp_generated_time = time.time()

    # Send the OTP to the recipient's email address.
    is_otp_sent, error = send_otp_via_email(otp, sender_email, sender_password, recipient_email_address)
    if is_otp_sent:
        # Enable the OTP entry and submit buttons, and disable the send OTP button and recipient email entry.
        otp_entry.config(state="normal")
        submit_otp_button.config(state="normal")
        send_otp_button.config(state="disabled")
        recipient_email_entry.config(state="disabled")
        
        # Reset the count of invalid attempts for the recipient's email.
        recipient_email_attempts = 0
        
        # Display an informational message indicating that the OTP has been sent.
        messagebox.showinfo("OTP Sent", "OTP has been sent to the recipient's email. Please enter it below.")
    else:
        # Display an error message if OTP sending fails.
        messagebox.showerror("Error", f"Failed to send OTP: {error}.")
        root.destroy()  # Close the main application window.

def handle_verify_otp(otp_entry, root):
    """
    Handles the process of verifying the entered OTP.

    Args:
        otp_entry (tk.Entry): The entry widget for entering the OTP.
        root (tk.Tk): The main application window.

    Returns:
        None
    """
    global otp, otp_attempts

    # Check if the OTP has expired using the is_otp_expired function.
    if is_otp_expired():
        # Show an error message if the OTP has expired and close the application.
        messagebox.showerror("Expired OTP", "The OTP has expired. Exiting the application.")
        root.destroy()  # Close the main application window.
        return  # Exit the function if OTP is expired.

    # Retrieve the entered OTP from the entry widget.
    entered_otp = otp_entry.get()
    
    # Validate the format of the entered OTP.
    if not entered_otp.isdigit() or len(entered_otp) != 6:
        # Show an error message if the OTP is invalid (not 6 digits).
        messagebox.showerror("Invalid OTP", "Please enter a valid 6-digit OTP.")
        return  # Exit the function if OTP format is invalid.

    # Verify if the entered OTP matches the generated OTP.
    if verify_otp(otp, int(entered_otp)):
        # If the OTP is correct, reset the OTP attempts counter and show a success message.
        otp_attempts = 0
        messagebox.showinfo("Success", "OTP verified successfully!")
        root.destroy()  # Close the main application window.
    else:
        # If the OTP is incorrect, increment the attempts counter.
        otp_attempts += 1
        if otp_attempts >= max_attempts:
            # Show an error message and close the application if the maximum number of attempts is reached.
            messagebox.showerror("Failure", "Maximum OTP attempts reached. Exiting the application.")
            root.destroy()  # Close the main application window.
        else:
            # Show an error message with the number of remaining attempts if the OTP is incorrect.
            messagebox.showerror("Failure", f"Incorrect OTP. Attempts remaining: {max_attempts - otp_attempts}")

def main():
    global otp_entry, recipient_email_entry, send_otp_button, submit_otp_button, root

    # UI setup
    root = tk.Tk()
    root.title("OTP Authentication")
    root.geometry("1280x720")  # Set an optimal resolution
    root.resizable(True, True)  # Allow the window to be resized

    # Font settings
    font_label = ("Helvetica", 18)
    font_entry = ("Helvetica", 16)
    font_button = ("Helvetica", 16, "bold")

    # Sender email and password
    tk.Label(root, text="Enter Sender Email:", font=font_label).grid(row=0, column=0, padx=20, pady=20, sticky="e")
    sender_email_entry = tk.Entry(root, font=font_entry, width=35)
    sender_email_entry.grid(row=0, column=1, padx=20, pady=20)

    tk.Label(root, text="Enter Sender Password:", font=font_label).grid(row=1, column=0, padx=20, pady=20, sticky="e")
    sender_password_entry = tk.Entry(root, font=font_entry, width=35, show='*')
    sender_password_entry.grid(row=1, column=1, padx=20, pady=20)

    verify_button = tk.Button(root, text="Verify", font=font_button, command=lambda: handle_verify_credentials(sender_email_entry, sender_password_entry, verify_button, send_otp_button, recipient_email_entry, root))
    verify_button.grid(row=1, column=2, padx=20, pady=20)

    # Recipient email
    tk.Label(root, text="Enter Recipient Email:", font=font_label).grid(row=2, column=0, padx=20, pady=20, sticky="e")
    recipient_email_entry = tk.Entry(root, font=font_entry, width=35, state="disabled")
    recipient_email_entry.grid(row=2, column=1, padx=20, pady=20)

    send_otp_button = tk.Button(root, text="Send OTP", font=font_button, state="disabled", command=lambda: handle_send_otp(sender_email_entry.get(), sender_password_entry.get(), recipient_email_entry, otp_entry, send_otp_button, submit_otp_button, root))
    send_otp_button.grid(row=2, column=2, padx=20, pady=20)

    # OTP entry
    tk.Label(root, text="Enter OTP:", font=font_label).grid(row=3, column=0, padx=20, pady=20, sticky="e")
    otp_entry = tk.Entry(root, font=font_entry, width=35, state="disabled")
    otp_entry.grid(row=3, column=1, padx=20, pady=20)

    submit_otp_button = tk.Button(root, text="Submit OTP", font=font_button, state="disabled", command=lambda: handle_verify_otp(otp_entry, root))
    submit_otp_button.grid(row=3, column=2, padx=20, pady=20)

    # Center the widgets in the window
    root.grid_columnconfigure(0, weight=1)
    root.grid_columnconfigure(1, weight=1)
    root.grid_columnconfigure(2, weight=1)
    root.grid_rowconfigure(0, weight=1)
    root.grid_rowconfigure(1, weight=1)
    root.grid_rowconfigure(2, weight=1)
    root.grid_rowconfigure(3, weight=1)

    root.mainloop()

if __name__ == "__main__":
    main()

Validation Error: Email must contain exactly one '@' symbol.
OTP sent successfully.
